In [1]:
import warnings
warnings.filterwarnings("ignore")

import os
import sys

dir2 = os.path.abspath('')
dir1 = os.path.dirname(dir2)
if not dir1 in sys.path:
    sys.path.append(dir1)

os.chdir('..')

%load_ext autoreload
%autoreload

In [2]:
from pathlib import Path

import numpy as np
import pandas as pd

import torch

from hydra import initialize, compose
from hydra.utils import instantiate, call

from pytorch_lightning import Trainer, seed_everything

from src.local_validation.local_validation_model import LocalValidationModelBase
from src.preprocessing import preprocess

from ptls.frames import PtlsDataModule

from src.datasets.coles import CustomColesDataset
from src.modules.coles import CustomCoLES

from src.pooling import PoolingModel

from torch.utils.data import Dataset
from src.utils.create_trainer import create_trainer
from typing import Union
from src.modules import Cotic, CustomCoLES, TS2Vec, VanillaAE
from pytorch_lightning.utilities.model_helpers import is_overridden
import torch.nn as nn

In [3]:
from src import learn_pooling

In [4]:
config_name = "master"
backbone_path = "saved_models/ar_default_1.pth"#"saved_models/churn/gpt/gpt_churn_1.pth"
SEED = 1

with initialize(config_path="../config", version_base=None):
    cfg = compose(config_name=config_name)
cfg_preprop = cfg["preprocessing"]
cfg_encoder = cfg["backbone"]["encoder"]

In [11]:
len(pd.DataFrame(train)["user_id"].unique())

5664

# Extra training

In [ ]:
train, val, test = preprocess(cfg_preprop)

sequence_encoder = instantiate(cfg_encoder, is_reduce_sequence=True)
sequence_encoder.load_state_dict(torch.load(backbone_path, map_location='cuda:0'))

pooling_model = PoolingModel(train_data=train,
                backbone=sequence_encoder,
                backbone_embd_size=sequence_encoder.seq_encoder.hidden_size,
                max_users_in_train_dataloader=300,
                pooling_type="learnable_attention",
                min_seq_length=15,
                max_seq_length=100,
                max_embs_per_user=500)

In [7]:
pooling_model.set_pooling_type("exp_learnable_hawkes", additional_config={"hidden_sizes": [100, 100]})# "learnable_attention")

In [8]:
# pooling_model.set_pooling_type("symmetrical_attention", additional_config={"hidden_size": 100})

In [9]:
# pooling_model.set_pooling_type("learnable_hawkes", additional_config={"hawkes_nn":{"hidden_sizes": [100, 100]},
#                                                                       "hawkes_time_nn":{"hidden_sizes": [10, 10]}})# "learnable_attention")

In [10]:
# pooling_model.set_pooling_type("kernel_attention", additional_config={"hidden_sizes": [100, 100]})

In [11]:
module = learn_pooling((train, val, test), pooling_model, learn_config_name="master", seed=SEED, validate=False)

Global seed set to 1


Data preprocessing...
Module creating...
Freezing of weights...
backbone.trx_encoder.embeddings.mcc_code.weight False
backbone.trx_encoder.numerical_batch_norm.bn.weight False
backbone.trx_encoder.numerical_batch_norm.bn.bias False
backbone.seq_encoder.starter_h False
backbone.seq_encoder.rnn.weight_ih_l0 False
backbone.seq_encoder.rnn.weight_hh_l0 False
backbone.seq_encoder.rnn.bias_ih_l0 False
backbone.seq_encoder.rnn.bias_hh_l0 False
learnable_attention_matrix.weight True
learnable_attention_matrix.bias True
hawkes_nn.layers.0.weight True
hawkes_nn.layers.0.bias True
hawkes_nn.layers.1.weight True
hawkes_nn.layers.1.bias True
hawkes_nn.layers.2.weight True
hawkes_nn.layers.2.bias True
Trainer creating...


wandb: Currently logged in as: makovka0911 (skoltech_dl_course). Use `wandb login --relogin` to force relogin


Using 16bit native Automatic Mixed Precision (AMP)
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name             | Type             | Params
------------------------------------------------------
0 | encoder          | PoolingModel     | 5.7 M 
1 | decoder          | AbsDecoder       | 0     
2 | amount_head      | Linear           | 2.0 K 
3 | mcc_head         | Linear           | 204 K 
4 | mcc_criterion    | CrossEntropyLoss | 0     
5 | amount_criterion | MSELoss          | 0     
6 | train_metrics    | ModuleDict       | 0     
7 | val_metrics      | ModuleDict       | 0     
8 | test_metrics     | ModuleDict       | 0     
------------------------------------------------------
1.6 M     Trainable params
4.3 M     Non-trainable params
5.9 M     Total params
11.778    Total estimated model params size (MB)


Training...


Training: 0it [00:00, ?it/s]

No `val_dataloader()` method defined to run `Trainer.validate`.


In [12]:
pooling_model = module.encoder

In [ ]:
pooling_model.backbone.is_reduce_sequence = True

In [ ]:
from src.global_validation.global_validation_pipeline import embed_data, eval_embeddings
seed_everything(SEED)
# validation

with initialize(config_path="../config", version_base=None):
    cfg = compose(config_name="master")

for val_type in cfg["validation"]:

    print(f"-------Validation {val_type}-------")
    cfg_validation = cfg["validation"][val_type]

    if val_type == "global_target":

        # get representations of sequences from train + val part
        embeddings, targets = embed_data(pooling_model, train + val, **cfg_validation["embed_data"])
        N = len(embeddings)
        indices = np.arange(N)

        # get representations of sequences from test part
        embeddings_test, targets_test = embed_data(
            pooling_model,
            test,
            **cfg_validation["embed_data"]
        )

        results = []
        # bootstrap sample
        bootstrap_inds = np.random.choice(indices, size=N, replace=True)
        embeddings_train, targets_train = (
            embeddings[bootstrap_inds],
            targets[bootstrap_inds],
        )
    
        # evaluate trained model
        metrics = eval_embeddings(
            embeddings_train,
            targets_train,
            embeddings_test,
            targets_test,
            cfg_validation["model"],
        )

        print(metrics)

    else:
        val_trainer = Trainer(**cfg_validation["trainer"])#,
                                #accelerator="gpu")
        
        valid_model: LocalValidationModelBase = instantiate(
        cfg_validation["module"],
            backbone=pooling_model,
            backbone_embd_size=pooling_model.embedding_size
        )

        train_dataset = call(cfg_validation["dataset"], data=train, deterministic=False)
        val_dataset = call(cfg_validation["dataset"], data=val, deterministic=True)
        test_dataset = call(cfg_validation["dataset"], data=test, deterministic=True)

        datamodule: PtlsDataModule = instantiate(
            cfg_validation["datamodule"],
            train_data=train_dataset,
            valid_data=val_dataset,
            test_data=test_dataset,
            )

        val_trainer.fit(valid_model, datamodule)
        val_trainer.test(valid_model, datamodule)